In [1]:
import pandas as pd
import numpy as np
import pickle
import os

/home/lbader/.conda/envs/py310-dm-tf-cloned/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.0' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
from my_functions.functions_feature_selection_ECML import MasterDataBuilder, FeatureAggregator

# Load data

## county information

In [3]:
path = 'data/original_data/pkl/county_information.pkl'
with open(path, 'rb') as file:
    county_information = pickle.load(file)
print(county_information['FIPS'].nunique())
county_information.head()

3233


,statefp,countyfp,countyns,FIPS,county_name,county_name_long,state_iso2,lsad,classfp,mtfcc,...,metdivfp,funcstat,aland,awater,latitude,longitude,state_name,GEOID,Region,Population_2023
0,39,063,01074044,39063,Hancock,Hancock County,OH,06,H1,G4020,...,None,A,1376125660,6020647,+41.0004711,-083.6660335,Ohio,39,North-Eastern,74704.0
1,51,179,01480173,51179,Stafford,Stafford County,VA,06,H1,G4020,...,47894,A,697237202,28088011,+38.4132608,-077.4513342,Virginia,51,Other,165428.0
2,31,181,00835912,31181,Webster,Webster County,NE,06,H1,G4020,...,None,A,1489017962,337241,+40.1806461,-098.4985897,Nebraska,31,Other,3351.0
3,48,127,01383849,48127,Dimmit,Dimmit County,TX,06,H1,G4020,...,None,A,3441795367,14576535,+28.4235871,-099.7658713,Texas,48,The Southern,8257.0
4,19,031,00465205,19031,Cedar,Cedar County,IA,06,H1,G4020,...,None,A,1500798712,6385832,+41.7723600,-091.1326105,Iowa,19,Other,18302.0


# Build master_df

In [4]:
df = pd.read_pickle('data/processed_data/pkl/df_cleaned.pickle')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2185008 entries, 0 to 2185007
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   FIPS                    object 
 1   statefp                 object 
 2   NAICS_CODE              object 
 3   NAICS_TITLE             object 
 4   emp_total_county_naics  int64  
 5   OCC_CODE                object 
 6   OCC_TITLE               object 
 7   emp_occupation          float64
 8   state_name              object 
dtypes: float64(1), int64(1), object(7)
memory usage: 150.0+ MB


In [5]:
df = pd.read_pickle('data/processed_data/pkl/df_pattern_4d_filtered.pickle')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186567 entries, 0 to 237799
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   fipstate      186567 non-null  int64 
 1   fipscty       186567 non-null  int64 
 2   naics         186567 non-null  object
 3   emp_nf        186567 non-null  object
 4   emp           186567 non-null  int64 
 5   qp1_nf        186567 non-null  object
 6   qp1           186567 non-null  int64 
 7   ap_nf         186567 non-null  object
 8   ap            186567 non-null  int64 
 9   est           186567 non-null  int64 
 10  n<5           186567 non-null  object
 11  n5_9          186567 non-null  object
 12  n10_19        186567 non-null  object
 13  n20_49        186567 non-null  object
 14  n50_99        186567 non-null  object
 15  n100_249      186567 non-null  object
 16  n250_499      186567 non-null  object
 17  n500_999      186567 non-null  object
 18  n1000         186567 non-null

In [11]:
# Deine Datei-Pfade
occ_path = 'data/processed_data/pkl/df_cleaned.pickle'
naics_path = 'data/processed_data/pkl/df_pattern_4d_filtered.pickle'

# Klasse initialisieren
builder = MasterDataBuilder(occ_path, naics_path, county_information)

# Code-Listen zusammenführen
combined_occ = ["51-9022", "51-4121", "49-9041", "49-9071", "51-4033", "51-4035", "47-2211", "51-2041"]
combined_naics = ["3315", "3361", "3362", "3364", "3366", "3335", "3320A1", "3320A2", "3327", "3312", "3314"]

# Aufruf
master_df = builder.build_master_df(combined_occ, combined_naics, 'data/master_df.pkl')
master_df['total_emp_naics_Automotive'] = master_df['total_emp_naics_3361'] + master_df['total_emp_naics_3362']

# Entfernen der alten Spalten
master_df = master_df.drop(columns=['total_emp_naics_3361', 'total_emp_naics_3362'])
master_df.to_pickle("data/processed_data/pkl/feature_df.pkl")
master_df

master_df successfully saved as a Pickle file at 'data/master_df.pkl'


,FIPS,total_emp_occu_51-9022,total_emp_occu_51-4121,total_emp_occu_49-9041,total_emp_occu_49-9071,total_emp_occu_51-4033,total_emp_occu_51-4035,total_emp_occu_47-2211,total_emp_occu_51-2041,total_emp_naics_3315,total_emp_naics_3364,total_emp_naics_3366,total_emp_naics_3335,total_emp_naics_3320A1,total_emp_naics_3320A2,total_emp_naics_3327,total_emp_naics_3312,total_emp_naics_3314,total_emp_naics_Automotive
0,39063,7.139367,112.109221,163.931470,606.245802,84.212192,14.770825,14.243830,5.062925,0.0,0.0,0.0,758.0,0.0,65.0,206.0,0.0,0.0,0.0
1,51179,0.701612,38.190271,79.435461,421.674025,0.216403,0.383702,37.487879,2.604025,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0
2,31181,0.000000,0.383612,1.766290,1.938639,0.005345,0.078293,0.000000,0.030081,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,48127,0.000000,22.729756,44.719147,23.669696,0.009005,0.000000,0.034805,0.086910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19031,0.063310,22.119571,22.357401,26.755142,0.766769,0.506321,5.864089,3.488307,0.0,0.0,0.0,0.0,0.0,52.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,28065,0.000000,13.637315,2.428384,10.538679,0.000000,0.000000,0.572519,0.075958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3229,47047,1.603672,40.724594,23.469043,90.820124,3.819688,2.863409,18.743847,7.195920,0.0,0.0,0.0,0.0,0.0,96.0,20.0,0.0,0.0,0.0
3230,31107,0.000000,0.733867,0.441573,14.111057,0.000000,0.058720,0.536157,0.060162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3231,41067,0.000000,1267.645029,1528.357399,2846.697115,215.565360,41.355708,450.490577,265.028336,0.0,10.0,0.0,274.0,402.0,800.0,1398.0,0.0,0.0,0.0


In [10]:
master_df

,FIPS,total_emp_occu_51-9022,total_emp_occu_51-4121,total_emp_occu_49-9041,total_emp_occu_49-9071,total_emp_occu_51-4033,total_emp_occu_51-4035,total_emp_occu_47-2211,total_emp_occu_51-2041,total_emp_naics_3315,total_emp_naics_3361,total_emp_naics_3362,total_emp_naics_3364,total_emp_naics_3366,total_emp_naics_3335,total_emp_naics_3320A1,total_emp_naics_3320A2,total_emp_naics_3327,total_emp_naics_3312,total_emp_naics_3314
0,39063,7.139367,112.109221,163.931470,606.245802,84.212192,14.770825,14.243830,5.062925,0.0,0.0,0.0,0.0,0.0,758.0,0.0,65.0,206.0,0.0,0.0
1,51179,0.701612,38.190271,79.435461,421.674025,0.216403,0.383702,37.487879,2.604025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0
2,31181,0.000000,0.383612,1.766290,1.938639,0.005345,0.078293,0.000000,0.030081,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,48127,0.000000,22.729756,44.719147,23.669696,0.009005,0.000000,0.034805,0.086910,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19031,0.063310,22.119571,22.357401,26.755142,0.766769,0.506321,5.864089,3.488307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,28065,0.000000,13.637315,2.428384,10.538679,0.000000,0.000000,0.572519,0.075958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3229,47047,1.603672,40.724594,23.469043,90.820124,3.819688,2.863409,18.743847,7.195920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96.0,20.0,0.0,0.0
3230,31107,0.000000,0.733867,0.441573,14.111057,0.000000,0.058720,0.536157,0.060162,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3231,41067,0.000000,1267.645029,1528.357399,2846.697115,215.565360,41.355708,450.490577,265.028336,0.0,0.0,0.0,10.0,0.0,274.0,402.0,800.0,1398.0,0.0,0.0


In [7]:
master_df.columns

Index(['FIPS', 'total_emp_occu_51-9022', 'total_emp_occu_51-4121',
       'total_emp_occu_49-9041', 'total_emp_occu_49-9071',
       'total_emp_occu_51-4033', 'total_emp_occu_51-4035',
       'total_emp_occu_47-2211', 'total_emp_occu_51-2041',
       'total_emp_naics_3315', 'total_emp_naics_3361', 'total_emp_naics_3362',
       'total_emp_naics_3364', 'total_emp_naics_3366', 'total_emp_naics_3335',
       'total_emp_naics_3320A1', 'total_emp_naics_3320A2',
       'total_emp_naics_3327', 'total_emp_naics_3312', 'total_emp_naics_3314'],
      dtype='object')

## Auswahl Plan B

In [ ]:
must_have = ['emp']  # Muss enthalten sein
list1 = occupations_plan_b  # ODER-Bedingung 1
list2 = naics_plan_b  # ODER-Bedingung 2


# Spalten filtern: Muss 'emp' enthalten und entweder aus list1 oder list2 etwas enthalten
selected_columns = ['FIPS'] + [
    col for col in master_df.columns
    if col != 'FIPS' and any(word in col for word in must_have) and (any(word in col for word in list1) or any(word in col for word in list2))
]

# Gefilterten DataFrame anzeigen
df_selected = master_df[selected_columns]
df_selected['total_emp_naics_Automotive'] = df_selected['total_emp_naics_3361'] + df_selected['total_emp_naics_3362']

# Entfernen der alten Spalten
df_selected = df_selected.drop(columns=['total_emp_naics_3361', 'total_emp_naics_3362'])
df_selected.to_pickle("data/processed_data/pkl/plan_b_df.pkl")
df_selected

In [ ]:
# This code defines a dictionary 'aggregate_columns' that maps column names to the corresponding columns in 'master_df' 
# and specifies the aggregation method ('sum'). The dictionary includes aggregations for different employee and establishment 
# data, divided by different occupation and NAICS categories.
# The 'FeatureAggregator' class is then instantiated with the 'master_df' and a path to save the final dataframe.
# The 'aggregate_columns' method of the 'FeatureAggregator' object is called to perform the aggregation based on the defined columns,
# and the resulting aggregated dataframe is stored in the 'result_df' variable.


aggregate_columns = {
    'emp_top10_to_20': (master_df.columns[11:21].to_list(), 'sum'),
    'total_emp_naics_top6': ([col for col in master_df.columns[21:33] if 'total_emp_naics' in col], 'sum'),
    'total_est_naics_top6': ([col for col in master_df.columns[21:33] if 'total_est_naics' in col], 'sum'),
    'total_emp_naics_metal': ([col for col in master_df.columns[33:53] if 'total_emp_naics' in col], 'sum'),
    'total_est_naics_metal': ([col for col in master_df.columns[33:53] if 'total_est_naics' in col], 'sum'),
}

aggregator = FeatureAggregator(master_df, 'data/processed_data/pkl/final_df.pickle')
result_df = aggregator.aggregate_columns(aggregate_columns)
result_df